In [ ]:
project_path = "/home/jupyter"
import os
import sys

sys.path.append(project_path)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from google.cloud import bigquery

from fintrans_toolbox.src import bq_utils as bq
from fintrans_toolbox.src import table_utils as t


client = bigquery.Client()

In [ ]:
# Summarise the data by UK Cardholder Abroad Spending All

UK_spending_by_Intl_All = '''SELECT time_period_value, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcc = 'All' 
and mcg = 'All' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM' 
GROUP BY destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_Intl_All = bq.read_bq_table_sql(client, UK_spending_by_Intl_All)
df_by_Intl_All = df_by_Intl_All.rename(columns={'spend': 'abroad_spend_all'})
df_by_Intl_All.head()

# Extract the year from the time_period_value (assuming it's in format like '2023-Q1')
df_by_Intl_All['year'] = df_by_Intl_All['time_period_value'].str[:4].astype(int)

# Group by year and sum the online spend for each year
df_yearly_Intl_All = df_by_Intl_All.groupby('year')['abroad_spend_all'].sum().reset_index()

# Optionally, sort the result by year
df_yearly_Intl_All = df_yearly_Intl_All.sort_values(by='year')

# Display the result
print(df_yearly_Intl_All)

df_yearly_Intl_All.to_csv('yearly_Intl_All.csv')

In [ ]:
# UK Cardholder Services Spending Abroad Online All

UK_spending_by_mccservices = '''SELECT 
    mcc, 
    SUM(spend) AS total_spend, 
    (SUM(spend) * 100.0 / (SELECT SUM(spend)  
                                          FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
                                          WHERE mcc IN ( 
'ACCOUNTANTS/AUDITORS/BOOKPR',
'ADVERTISING SERVICES',
'AQUARIUMS/SEAQUARIUMS',
'ARCHITECTURAL/ENG/SURVEY',
'AUTO BODY REPAIR SHOPS',
'AUTO PAINT SHOPS',
'AUTO SERVICE SHOPS/NON DEALER',
'BANDS/ORCHESTRAS/ENTERTAIN',
'BEAUTY/BARBER SHOPS',
'BILLIARD/POOL ESTABLISHMENT',
'BOWLING ALLEYS',
'BUS LINES',
'BUSINESS SERVICES - DEFAULT',
'BUSINESS/SECRETARIAL SCHOOL',
'CABLE, SAT, PAY TV/RADIO SVCS',
'CATERERS', 
'COLLEGES/UNIV/JC/PROFESSION',
'COMPUTER MAINT/SVCS - DEF',
'COMPUTER NETWORK/INFO SVCS',
'COMPUTER PROGRAM/SYS DESIGN',
'CONTRACTORS - CONCRETE',
'CORRESPONDENCE SCHOOLS',
'COURIER SERVICES',
'DANCE HALLS/STUDIOS/SCHOOLS',
'DETECTIVE/PROTECTIVE AGEN',
'DIGITAL GOODS APP(EXCL GAMES)',
'DIRECT SELL/DOOR-TO-DOOR',
'DRY CLEANERS',
'ELECTRICAL CONTRACTORS',
'EMPLOYMENT/TEMP HELP AGEN',
'EXTERMINATING/DISINFECT SERV',
'FURNITURE REUPHOLSTERY/REPAIR',
'GEN CONTRACTORS RESIDENTL/COML',
'HEALTH CARE',
'HEALTH & BEAUTY SPAS',
'INFORMATION RETRIEVAL SERVICES',
'INTRA-GOVERNMENT PURCHASES',
'LANDSCAPE/HORTICULTURAL SER',
'LAUNDRIES-FAMILY/COMMERCIAL',
'LAUNDRY/CLEANING/GARMENT SV',
'LOCAL COMMUTER TRANSPORT',
'MARINAS, SERVICE & SUPPLY',
'MEMBER CLUBS/SPORT/REC/GOLF',
'MISC PERSONAL SERV - DEF',
'MISC REPAIR SERVICES',
'MOTION PICTURE THEATRES',
'MOTOR FREIGHT CARRIERS',
'OUTBOUND TELEMARKETING MERCHNT',
'PARKING LOTS,METERS,GARAGES',
'PASSENGER RAILWAYS',
'PHOTO FINISH LABS/DEV',
'PROFESSIONAL SERVICES - DEF',
'PUBLIC GOLF COURSES',
'PUBLIC WAREHOUSING',
'RADIO/TV/STEREO REPAIR SHOP',
'RAILROADS',
'REAL EST AGNTS & MGRS RENTALS',
'RECREATION SERVICES',
'SHOE REPAIR/SHINE/HAT CLEAN',
'SMALL APPLIANCE REPAIR DEF',
'SPECIALTY CLEANING/POLISHING',
'SPORTING/RECREATIONAL CAMPS',
'STENOGRAPHIC SERVICES',
'TAX PAYMENTS',
'TAX PREPARATION SERVICE',
'TAXICABS/LIMOUSINES',
'TELECOMMUNICATION SERVICES',
'TELEGRAPH SERVICES',
'TESTING LABS (NON-MEDICAL)',
'THEATRICAL PRODUCERS',
'TIRE RETREAD/REPAIR SHOPS',
'TOLLS AND BRIDGE FEES',
'TOURIST ATTRACTIONS AND XHBT',
'TOWING SERVICES',
'TRADE/VOCATIONAL SCHOOLS',
'TRANSPORTATION SVCS - DEFAULT',
'TRAVEL AGENCIES',
'TYPESETTING/PLATE MAKING ETC',
'TYPEWRITER/SALES/SERVICE',
'VETERINARY SERVICES',
'WATCH/CLOCK/JEWELRY REPAIR',
'WELDING SERVICES',
       'VIDEO GAME ARCADES/ESTABLISH'
 ) ) ) AS b2b_services_percentage 
FROM  
    `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
WHERE  
    mcc IN ( 
       'ACCOUNTANTS/AUDITORS/BOOKPR',
'ADVERTISING SERVICES',
'AQUARIUMS/SEAQUARIUMS',
'ARCHITECTURAL/ENG/SURVEY',
'AUTO BODY REPAIR SHOPS',
'AUTO PAINT SHOPS',
'AUTO SERVICE SHOPS/NON DEALER',
'BANDS/ORCHESTRAS/ENTERTAIN',
'BEAUTY/BARBER SHOPS',
'BILLIARD/POOL ESTABLISHMENT',
'BOWLING ALLEYS',
'BUS LINES',
'BUSINESS SERVICES - DEFAULT',
'BUSINESS/SECRETARIAL SCHOOL',
'CABLE, SAT, PAY TV/RADIO SVCS',
'CATERERS', 
'COLLEGES/UNIV/JC/PROFESSION',
'COMPUTER MAINT/SVCS - DEF',
'COMPUTER NETWORK/INFO SVCS',
'COMPUTER PROGRAM/SYS DESIGN',
'CONTRACTORS - CONCRETE',
'CORRESPONDENCE SCHOOLS',
'COURIER SERVICES',
'DANCE HALLS/STUDIOS/SCHOOLS',
'DETECTIVE/PROTECTIVE AGEN',
'DIGITAL GOODS APP(EXCL GAMES)',
'DIRECT SELL/DOOR-TO-DOOR',
'DRY CLEANERS',
'ELECTRICAL CONTRACTORS',
'EMPLOYMENT/TEMP HELP AGEN',
'EXTERMINATING/DISINFECT SERV',
'FURNITURE REUPHOLSTERY/REPAIR',
'GEN CONTRACTORS RESIDENTL/COML',
'HEALTH CARE',
'HEALTH & BEAUTY SPAS',
'INFORMATION RETRIEVAL SERVICES',
'INTRA-GOVERNMENT PURCHASES',
'LANDSCAPE/HORTICULTURAL SER',
'LAUNDRIES-FAMILY/COMMERCIAL',
'LAUNDRY/CLEANING/GARMENT SV',
'LOCAL COMMUTER TRANSPORT',
'MARINAS, SERVICE & SUPPLY',
'MEMBER CLUBS/SPORT/REC/GOLF',
'MISC PERSONAL SERV - DEF',
'MISC REPAIR SERVICES',
'MOTION PICTURE THEATRES',
'MOTOR FREIGHT CARRIERS',
'OUTBOUND TELEMARKETING MERCHNT',
'PARKING LOTS,METERS,GARAGES',
'PASSENGER RAILWAYS',
'PHOTO FINISH LABS/DEV',
'PROFESSIONAL SERVICES - DEF',
'PUBLIC GOLF COURSES',
'PUBLIC WAREHOUSING',
'RADIO/TV/STEREO REPAIR SHOP',
'RAILROADS',
'REAL EST AGNTS & MGRS RENTALS',
'RECREATION SERVICES',
'SHOE REPAIR/SHINE/HAT CLEAN',
'SMALL APPLIANCE REPAIR DEF',
'SPECIALTY CLEANING/POLISHING',
'SPORTING/RECREATIONAL CAMPS',
'STENOGRAPHIC SERVICES',
'TAX PAYMENTS',
'TAX PREPARATION SERVICE',
'TAXICABS/LIMOUSINES',
'TELECOMMUNICATION SERVICES',
'TELEGRAPH SERVICES',
'TESTING LABS (NON-MEDICAL)',
'THEATRICAL PRODUCERS',
'TIRE RETREAD/REPAIR SHOPS',
'TOLLS AND BRIDGE FEES',
'TOURIST ATTRACTIONS AND XHBT',
'TOWING SERVICES',
'TRADE/VOCATIONAL SCHOOLS',
'TRANSPORTATION SVCS - DEFAULT',
'TRAVEL AGENCIES',
'TYPESETTING/PLATE MAKING ETC',
'TYPEWRITER/SALES/SERVICE',
'VETERINARY SERVICES',
'WATCH/CLOCK/JEWELRY REPAIR',
'WELDING SERVICES',
       'VIDEO GAME ARCADES/ESTABLISH'
) 
and time_period = 'Quarter' 
and merchant_channel = 'Online' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM'
GROUP BY  
    mcc
ORDER BY  
    total_spend DESC'''
df_by_mccservices = bq.read_bq_table_sql(client, UK_spending_by_mccservices)     
df_by_mccservices

In [ ]:
df_by_mccservices.to_csv('UK_spending_by_mccservices.csv')

In [ ]:
# Bar Chart UK Cardholder Services Spending Abroad Online All

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure the data is in the correct format
# Assuming df_by_mccgoods is your DataFrame from the SQL query result

# Sort data by 'total_spend' for better visualization
df_by_mccservices = df_by_mccservices.sort_values(by='total_spend', ascending=False)

# Set the plot size for readability
plt.figure(figsize=(12, 8))

# Create a bar chart with total spend
sns.barplot(x='total_spend', y='mcc', data=df_by_mccservices, palette='viridis')

# Add labels and title
plt.xlabel('Total Spend (£)', fontsize=14)
plt.ylabel('Merchant Category Code (MCC)', fontsize=14)
plt.title('Total Spend by MCC (Online)', fontsize=16)

# Add a second axis to show B2B percentage
# If you want to visualize both total spend and B2B percentage in a dual-axis chart, you can do something like this:

fig, ax1 = plt.subplots(figsize=(12, 8))

# Plot total spend
sns.barplot(x='total_spend', y='mcc', data=df_by_mccservices, palette='Blues', ax=ax1)
ax1.set_xlabel('Total Spend (£)', fontsize=14)
ax1.set_ylabel('Merchant Category Code (MCC)', fontsize=14)
ax1.set_title('Total Spend and B2B Services Percentage by MCC (Online Abroad)', fontsize=16)

# Create a second axis for B2B percentage
ax2 = ax1.twiny()
sns.lineplot(x='b2b_services_percentage', y='mcc', data=df_by_mccservices, color='red', marker='o', ax=ax2)
ax2.set_xlabel('B2B Services Percentage (%)', fontsize=14, color='red')
ax2.tick_params(axis='x', labelcolor='red')

# Display the plot
plt.show()

In [ ]:
# Yearly UK Cardholder Services Spending Abroad Online All


UK_spending_by_mccservices_yearly = '''
SELECT 
    CAST(SUBSTR(time_period_value, 1, 4) AS INT64) AS year,  -- Extract the first 4 characters (year) from time_period_value
    SUM(spend) AS mcc_services_total  -- Sum the total spend for each year
FROM  
    `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
WHERE  
    mcc IN ( 
   'ACCOUNTANTS/AUDITORS/BOOKPR',
'ADVERTISING SERVICES',
'AQUARIUMS/SEAQUARIUMS',
'ARCHITECTURAL/ENG/SURVEY',
'AUTO BODY REPAIR SHOPS',
'AUTO PAINT SHOPS',
'AUTO SERVICE SHOPS/NON DEALER',
'BANDS/ORCHESTRAS/ENTERTAIN',
'BEAUTY/BARBER SHOPS',
'BILLIARD/POOL ESTABLISHMENT',
'BOWLING ALLEYS',
'BUS LINES',
'BUSINESS SERVICES - DEFAULT',
'BUSINESS/SECRETARIAL SCHOOL',
'CABLE, SAT, PAY TV/RADIO SVCS',
'CATERERS', 
'COLLEGES/UNIV/JC/PROFESSION',
'COMPUTER MAINT/SVCS - DEF',
'COMPUTER NETWORK/INFO SVCS',
'COMPUTER PROGRAM/SYS DESIGN',
'CONTRACTORS - CONCRETE',
'CORRESPONDENCE SCHOOLS',
'COURIER SERVICES',
'DANCE HALLS/STUDIOS/SCHOOLS',
'DETECTIVE/PROTECTIVE AGEN',
'DIGITAL GOODS APP(EXCL GAMES)',
'DIRECT SELL/DOOR-TO-DOOR',
'DRY CLEANERS',
'ELECTRICAL CONTRACTORS',
'EMPLOYMENT/TEMP HELP AGEN',
'EXTERMINATING/DISINFECT SERV',
'FURNITURE REUPHOLSTERY/REPAIR',
'GEN CONTRACTORS RESIDENTL/COML',
'HEALTH CARE',
'HEALTH & BEAUTY SPAS',
'INFORMATION RETRIEVAL SERVICES',
'INTRA-GOVERNMENT PURCHASES',
'LANDSCAPE/HORTICULTURAL SER',
'LAUNDRIES-FAMILY/COMMERCIAL',
'LAUNDRY/CLEANING/GARMENT SV',
'LOCAL COMMUTER TRANSPORT',
'MARINAS, SERVICE & SUPPLY',
'MEMBER CLUBS/SPORT/REC/GOLF',
'MISC PERSONAL SERV - DEF',
'MISC REPAIR SERVICES',
'MOTION PICTURE THEATRES',
'MOTOR FREIGHT CARRIERS',
'OUTBOUND TELEMARKETING MERCHNT',
'PARKING LOTS,METERS,GARAGES',
'PASSENGER RAILWAYS',
'PHOTO FINISH LABS/DEV',
'PROFESSIONAL SERVICES - DEF',
'PUBLIC GOLF COURSES',
'PUBLIC WAREHOUSING',
'RADIO/TV/STEREO REPAIR SHOP',
'RAILROADS',
'REAL EST AGNTS & MGRS RENTALS',
'RECREATION SERVICES',
'SHOE REPAIR/SHINE/HAT CLEAN',
'SMALL APPLIANCE REPAIR DEF',
'SPECIALTY CLEANING/POLISHING',
'SPORTING/RECREATIONAL CAMPS',
'STENOGRAPHIC SERVICES',
'TAX PAYMENTS',
'TAX PREPARATION SERVICE',
'TAXICABS/LIMOUSINES',
'TELECOMMUNICATION SERVICES',
'TELEGRAPH SERVICES',
'TESTING LABS (NON-MEDICAL)',
'THEATRICAL PRODUCERS',
'TIRE RETREAD/REPAIR SHOPS',
'TOLLS AND BRIDGE FEES',
'TOURIST ATTRACTIONS AND XHBT',
'TOWING SERVICES',
'TRADE/VOCATIONAL SCHOOLS',
'TRANSPORTATION SVCS - DEFAULT',
'TRAVEL AGENCIES',
'TYPESETTING/PLATE MAKING ETC',
'TYPEWRITER/SALES/SERVICE',
'VETERINARY SERVICES',
'WATCH/CLOCK/JEWELRY REPAIR',
'WELDING SERVICES',
       'VIDEO GAME ARCADES/ESTABLISH'
) 
 AND time_period = 'Quarter' 
    AND merchant_channel = 'Online' 
    and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM' 
and destination_country != 'UNITED KINGDOM'
    AND CAST(SUBSTR(time_period_value, 1, 4) AS INT64) BETWEEN 2019 AND 2025  -- Filter by year range
GROUP BY  
    year
ORDER BY  
    year'''

df_by_mccservices_yearly = bq.read_bq_table_sql(client, UK_spending_by_mccservices_yearly)
df_by_mccservices_yearly

df_by_mccservices_yearly.to_csv("UK_spending_by_mccservices_yearly.csv", index=False)


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file
csv_file = 'UK_spending_by_mccservices_yearly.csv'
df_services = pd.read_csv(csv_file)

# Create the bar trace
bar_trace = go.Bar(
    x=df_services['year'],
    y=df_services['mcc_services_total'] / 1e9,  # Convert values to billions
    name='MCC Services Abroad Online Total',
    text=(df_services['mcc_services_total'] / 1e9).round(2),  # Show values in billions with 2 decimal places
    hoverinfo='text+x+y',
    marker=dict(color='blue'),
    orientation='v'
)

# Create the line trace
line_trace = go.Scatter(
    x=df_services['year'],
    y=df_services['mcc_services_total'] / 1e9,
    mode='lines+markers',
    name='MCC Services Abroad Online Total (Line)',
    line=dict(color='orange', width=2),
    marker=dict(color='orange')
)

# Plotting the Bar and Line Chart
fig = go.Figure(data=[bar_trace, line_trace])

fig.update_layout(
    title="MCC Services Abroad Online Total (Billions GBP)",
    xaxis_title="Year",
    yaxis_title="Amount in Billions GBP",
    template="plotly_dark",
    xaxis=dict(tickmode='array', tickvals=df_services['year']),
    yaxis=dict(
        title="Amount in Billions GBP",
        showgrid=True,
        zeroline=False,
        tickformat='.0f'
    ),
    showlegend=True
)

# Display the combined bar and line chart
fig.show()



In [ ]:
# Summarise the data by UK Cardholder Spending All

UK_spending_All = '''SELECT time_period_value, destination_country, spend 
FROM `ons-fintrans-data-prod.fintrans_visa.spend_origin_and_channel` 
where time_period = 'Quarter' 
and mcc = 'All' 
and mcg = 'All' 
and merchant_channel = 'All' 
and cardholder_origin_country = 'All' 
and cardholder_origin = 'UNITED KINGDOM'  
GROUP BY destination_country, 
time_period_value, spend 
ORDER BY time_period_value, destination_country DESC'''
df_by_All = bq.read_bq_table_sql(client, UK_spending_All)
df_by_All.head()

#Caculate UK Total Spending

import pandas as pd

# Assuming df_by_Dom_All is the DataFrame with your data
# Ensure 'time_period_value' is a string type and split it to get the year (assuming 'Q1', 'Q2', etc., are part of the time_period_value)

# Extract the year from the time_period_value (assuming it's in the format like '2023-Q1', '2023-Q2', etc.)
df_by_All['year'] = df_by_All['time_period_value'].str[:4].astype(int)

# Now group by year and sum the spend for each year
df_yearly_All = df_by_All.groupby('year')['spend'].sum().reset_index()

# Optionally, you can sort the result by year
df_yearly_All = df_yearly_All.sort_values(by='year')

# Display the yearly totals
print(df_yearly_All)

df_yearly_All.to_csv('UK_spending_by_yearly_All.csv')

In [ ]:
# UK Cardholder Abroad spending ratio

import pandas as pd

# Load the data from CSV files
df_yearly_Intl_All = pd.read_csv("yearly_Intl_All.csv")
df_yearly_All = pd.read_csv("UK_spending_by_yearly_All.csv")

# Merge the two dataframes on 'year'
df_merged = pd.merge(df_yearly_All, df_yearly_Intl_All, on='year')

# Calculate the abroad spending ratio
df_merged['UK_Abroad_Spending_Ratio'] = df_merged['abroad_spend_all'] / df_merged['spend']

# Display the result
print(df_merged[['year', 'UK_Abroad_Spending_Ratio']])


# Save the result to a new CSV file
df_merged.to_csv("UK_Abroad_Spending_Ratio.csv", index=False)

# Display the result
df_merged


In [ ]:
# UK_Service_Abroad_Online_Spending_Ratio


import pandas as pd

# Load the two CSV files
df_services_total = pd.read_csv("UK_spending_by_mccservices_yearly.csv")
df_spending_visa = pd.read_csv("UK_spending_by_yearly_All.csv")

# Merge the dataframes on 'year'
df_combined = pd.merge(df_services_total, df_spending_visa, on='year', how='inner')

# Calculate the UK Service Abroad Online Spending Ratio
df_combined['UK_Service_Abroad_Online_Spending_Ratio'] = (
    df_combined['mcc_services_total'] / df_combined['spend'] * 100
)

# Display the resulting DataFrame with year and the calculated ratio
df_combined[['year', 'UK_Service_Abroad_Online_Spending_Ratio']]

# Save to CSV for later use
df_combined.to_csv('UK_Service_Abroad_Online_Spending_Ratio.csv', index=False)

In [ ]:
# UK Finance Total

project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)

from google.cloud import bigquery
import importlib
import plotly.express as px

import numpy as np
import pandas as pd
from datetime import datetime

import ft_digital_trade.src.utils.read_data as read_utils
import ft_digital_trade.src.utils.clean_utils as clean_utils
import ft_digital_trade.src.utils.calculation_utils as calc_utils
import ft_digital_trade.src.utils.plot_utils as plot_utils

client = bigquery.Client()

visa_data = read_utils.read_visa(cardholder_origin = "uk", cardholders_location = "uk", spend_location = "uk")

visa = calc_utils.calculate_visa(visa_data)
visa = clean_utils.rename_columns(df = visa, suffix = '_spoc')

global_cards = read_utils.read_global_cards()
global_cards = clean_utils.clean_global(global_cards)
global_cards = calc_utils.calculate_global(global_cards, 'card')

global_spend = read_utils.read_global_spend()
global_spend = clean_utils.clean_global(global_spend)
global_spend = calc_utils.calculate_global(global_spend, 'spend')

global_df = global_cards.merge(global_spend, how = 'inner', on = 'year', suffixes = ('_cards', '_spend'))
global_df = clean_utils.rename_columns(df = global_df, suffix = '_global')

uk_finance = read_utils.read_uk_finance()
uk_finance = clean_utils.clean_uk_finance(uk_finance)
uk_finance = calc_utils.calculate_uk_finance(uk_finance)
uk_finance = uk_finance[['year', 'cardholders','total value of purchases',"total volume of purchases"]]
uk_finance = clean_utils.rename_columns(df = uk_finance , suffix = '_uk_finance')

boe = read_utils.read_boe()
boe = clean_utils.clean_boe(boe)
boe = calc_utils.calculate_boe(boe)
boe = clean_utils.rename_columns(df = boe , suffix = '_boe')

link = read_utils.read_link()

merged = visa.merge(uk_finance, how = 'outer', on = 'year')
merged = merged.merge(boe, how = 'outer', on = 'year')
merged = merged.merge(global_df, how = 'outer', on = 'year')

cardholders = merged[['year','cardholders_spoc','cardholders_uk_finance','visa_total_cards_global','total_cards_global', 'visa_marketshare_cards_global']]
cardholders = cardholders.copy()
cardholders['uk_finance_marketshare'] = cardholders['cardholders_spoc'] / cardholders['cardholders_uk_finance'] *100
cardholders['global_marketshare'] = cardholders['cardholders_spoc'] / cardholders['total_cards_global'] *100
#melt df for charts
cardholders = pd.melt(cardholders, id_vars='year',var_name='Data source', value_name='value')
cardholders = calc_utils.calculate_index(df = cardholders)

spend = merged[['year','spend_spoc', 
        'total value of purchases_uk_finance',
       'Mastercard values_boe', 'Visa Europe values_boe',
       'Mastercard and Visa values_boe', 'Visa proportion_boe',
       'debit_spend_global', 'credit_spend_global', 'visa_total_spend_global',
       'total_spend_global', 'visa_marketshare_spend_global']]
spend = spend.copy()
# #replace 2024 spending with NA
spend['spend_spoc'] = np.where(spend['year']==2025, np.nan, spend['spend_spoc'])
spend['total value of purchases_uk_finance'] = np.where(spend['year']==2025, np.nan, spend['total value of purchases_uk_finance'])
#calculate marketshare
spend['uk_finance_marketshare'] = spend['spend_spoc'] / spend['total value of purchases_uk_finance'] *100
spend['global_marketshare'] = spend['spend_spoc'] / spend['total_spend_global'] *100
spend['boe_marketshare'] = spend['spend_spoc'] / spend['Mastercard and Visa values_boe'] *100
#copy used for getting 2019 marketshare
spend_copy = spend.copy()
#melt df for charts
spend = pd.melt(spend, id_vars='year',var_name='Data source', value_name='value')
spend = calc_utils.calculate_index(df = spend)

#options of marketshare threshold
# marketshare_2019 = spend_copy.iloc[0]['visa_marketshare_spend_global']
# marketshare_2019 = spend_copy.iloc[0]['global_marketshare']
marketshare_2019 = spend_copy.iloc[0]['uk_finance_marketshare']
marketshare_2019

df = merged.copy() 
#remove 2025 due to incomplete data
# df = df[df['year'] != 2025] ------------------------------- EXCL
#index spoc data
df['idx_cardholders_spoc'] = df['cardholders_spoc'].transform(lambda x: (x / x.iloc[0] * 100))
df['idx_spend_spoc'] = df['spend_spoc'].transform(lambda x: (x / x.iloc[0] * 100))
# adjust visa spend to 2019 cardholders (assume same number of cardholders each year)
df['visa_adj_spend_spoc'] = (df['spend_spoc']/df['idx_cardholders_spoc'])*100
df['total_spoc'] = df['visa_adj_spend_spoc'] / marketshare_2019 *100
#rename columns
df = df.rename(columns={'total value of purchases_uk_finance': 'total_uk_finance'})
df = df.rename(columns={'visa_total_spend_global': 'visa_total_global','total_spend_global':'total_global'})
df = df.rename(columns={'Visa Europe values_boe': 'visa_total_boe', 'Mastercard and Visa values_boe': 'total_boe'})
#filter columns
df = df[['year', 'visa_adj_spend_spoc', 'total_spoc', 'visa_total_global', 'total_global', 'total_uk_finance', 'visa_total_boe', 'total_boe' ]]
#melt df
df = pd.melt(df, id_vars='year',var_name='Data source', value_name='value')
df = calc_utils.calculate_index(df = df)

# total_uk_finance: Total spend in the UK finance sector.
# visa_total_global: Total global spend for Visa.
# total_global: Total global spend (presumably for both Visa and Mastercard).
# visa_total_boe: Total spend for Visa in the Bank of England (BOE) area.
# total_boe: Total spend in the BOE area, for both Visa and Mastercard.

# Save total_uk_finance separetely in a cvs file

import pandas as pd

# Assuming 'df' is already defined in the user's environment and contains the melted data
# Filter for 'total_uk_finance' entries
df_total_uk_finance = df[df['Data source'] == 'total_uk_finance'][['year', 'value']].rename(columns={'value': 'total_uk_finance'})

# Save to CSV for later use
df_total_uk_finance.to_csv('total_uk_finance.csv', index=False)

# Display the saved DataFrame
print(df_total_uk_finance)

In [ ]:
# UK_Finance_Abroad_Total

import pandas as pd

# Load the total UK finance data
df_total_finance = pd.read_csv('total_uk_finance.csv')

# Load the UK Abroad Spending Ratio data
df_spending_ratio = pd.read_csv('UK_Abroad_Spending_Ratio.csv')

# Merge the two dataframes on the 'year' column
df_merged = pd.merge(df_total_finance, df_spending_ratio, on='year')

# Calculate the UK Finance Abroad Total
df_merged['UK_Finance_Abroad_Total'] = df_merged['total_uk_finance'] * df_merged['UK_Abroad_Spending_Ratio']

# Display the resulting dataframe
df_merged[['year', 'UK_Finance_Abroad_Total']]

# Save to CSV for later use
df_merged.to_csv('UK_Finance_Abroad_Total.csv', index=False)

# Display the saved DataFrame
print(df_merged)

In [ ]:
# UK_Finance_Services_Abroad_Online_Spending_Total 

import pandas as pd

# Load the CSV files
df_finance_spend = pd.read_csv("total_uk_finance.csv")
df_service_ratio = pd.read_csv("UK_Service_Abroad_Online_Spending_Ratio.csv")

# Merge the two dataframes on 'year'
df_combined = pd.merge(df_finance_spend, df_service_ratio, on='year', how='inner')

# Calculate the UK Finance Services Abroad Online Spending Total
df_combined['UK_Finance_Services_Abroad_Online_Spending_Total'] = (
    df_combined['total_uk_finance'] * df_combined['UK_Service_Abroad_Online_Spending_Ratio'] / 100
)

# Display the results
df_combined[['year', 'UK_Finance_Services_Abroad_Online_Spending_Total']]

# Save to CSV for later use
df_combined.to_csv('UK_Finance_Services_Abroad_Online_Spending_Total.csv', index=False)

In [ ]:
# Bar Chart for UK_Finance_Services_Abroad_Online_Spending_Total 

import pandas as pd
import plotly.graph_objects as go

# Load the data from the CSV file
csv_file = 'UK_Finance_Services_Abroad_Online_Spending_Total.csv'
df_combined = pd.read_csv(csv_file)

# Create the bar trace
bar_trace = go.Bar(
    x=df_combined['year'],
    y=df_combined['UK_Finance_Services_Abroad_Online_Spending_Total'],
    name='Bar Chart - Spending',
    marker_color='green',
    text=df_combined['UK_Finance_Services_Abroad_Online_Spending_Total'],
    hoverinfo='text+x+y',
)

# Create the line trace
line_trace = go.Scatter(
    x=df_combined['year'],
    y=df_combined['UK_Finance_Services_Abroad_Online_Spending_Total'],
    mode='lines+markers',
    name='Line Chart - Spending',
    line=dict(color='blue', width=2),
    marker=dict(color='blue', size=6),
    text=df_combined['UK_Finance_Services_Abroad_Online_Spending_Total'],
    hoverinfo='text+x+y',
)

# Combine both the bar and line charts into one figure
fig = go.Figure(data=[bar_trace, line_trace])

# Update layout for better visualization
fig.update_layout(
    title="UK Finance Services Abroad Online Spending Total (2019-2025)",
    xaxis_title="Year",
    yaxis_title="Spending Amount (in GBP)",
    template="plotly_dark",
    barmode='group',
    showlegend=True,
    yaxis=dict(tickformat=".2e"),
)

# Show the figure
fig.show()



In [ ]:
# Bar Chart UK_Finance_Services_Abroad_Online_Spending_Total_Billions

import pandas as pd
import plotly.graph_objects as go

# Load the data from the CSV file
csv_file = 'UK_Finance_Services_Abroad_Online_Spending_Total.csv'
df_combined = pd.read_csv(csv_file)

# Convert values to billions (divide by 1e9) and round to 2 decimal places
df_combined['UK_Finance_Services_Abroad_Online_Spending_Total_Billions'] = (
    df_combined['UK_Finance_Services_Abroad_Online_Spending_Total'] / 1e9
).round(2)

# Create the bar trace
bar_trace = go.Bar(
    x=df_combined['year'],
    y=df_combined['UK_Finance_Services_Abroad_Online_Spending_Total_Billions'],
    name='Bar Chart - Spending',
    marker_color='green',
    text=df_combined['UK_Finance_Services_Abroad_Online_Spending_Total_Billions'],
    hoverinfo='text+x+y',
)

# Create the line trace
line_trace = go.Scatter(
    x=df_combined['year'],
    y=df_combined['UK_Finance_Services_Abroad_Online_Spending_Total_Billions'],
    mode='lines+markers',
    name='Line Chart - Spending',
    line=dict(color='blue', width=2),
    marker=dict(color='blue', size=6),
    text=df_combined['UK_Finance_Services_Abroad_Online_Spending_Total_Billions'],
    hoverinfo='text+x+y',
)

# Combine both the bar and line charts into one figure
fig = go.Figure(data=[bar_trace, line_trace])

# Update layout for better visualization
fig.update_layout(
    title="UK Finance Services Abroad Online Spending Total (2019-2025)",
    xaxis_title="Year",
    yaxis_title="Spending Amount (in Billions GBP)",
    template="plotly_dark",
    barmode='group',
    showlegend=True,
    yaxis=dict(
        tickformat=".0f",
        title="Amount in Billions GBP"
    ),
    xaxis=dict(tickmode='array', tickvals=df_combined['year']),
)

# Show the figure
fig.show()



In [ ]:
#UK Finance Abroad Total

   year  uk_abroad_total_for_uk_finance
0  2019                    4.277365e+10
1  2020                    4.389336e+10
2  2021                    4.415398e+10
3  2022                    3.782894e+10
4  2023                    4.095468e+10

#UK Finance Abroad Goods Online Total
   year  UK_Finance_Goods_Online_Abroad_Total_Spending
0  2019                                   1.647485e+10
1  2020                                   2.140762e+10
2  2021                                   2.017022e+10
3  2022                                   8.926549e+09
4  2023                                   9.445098e+09

#UK Finance Abroad Online Total				
		year	UK_Abroad_Online_Total	
		2019	3.05E+10	
		2020	3.67E+10	
		2021	3.65E+10	
		2022	2.41E+10	
		2023	2.51E+10	
        
        #UK Finance Abroad Services Online Total      
year	UK_Finance_Services_Online_Abroad_Total
2019	1.40E+10
2020	1.53E+10
2021	1.63E+10
2022	1.52E+10
2023	1.57E+10


In [ ]:
# Compare UK Finance Services Online and UK Abroad Spending 2019 - 2025

import pandas as pd
import plotly.express as px

# Load the CSV files
df_full = pd.read_csv("UK_Finance_Services_Abroad_Online_Spending_Total.csv")
df_total = pd.read_csv("UK_Finance_Abroad_Total.csv")

# Extract relevant columns
df_filtered = df_full[['year', 'UK_Finance_Services_Abroad_Online_Spending_Total']]
df_total_filtered = df_total[['year', 'UK_Finance_Abroad_Total']]

# Merge the dataframes on 'year'
df_merged = pd.merge(df_filtered, df_total_filtered, on='year')

# Create the line chart
fig_line = px.line(df_merged, 
                   x='year', 
                   y=['UK_Finance_Services_Abroad_Online_Spending_Total', 'UK_Finance_Abroad_Total'], 
                   title='UK Finance Services Online and UK Abroad Spending 2019 - 2025',
                   labels={
                       'year': 'Year', 
                       'UK_Finance_Services_Abroad_Online_Spending_Total': 'Services Online Abroad (£)', 
                       'UK_Finance_Abroad_Total': 'UK Abroad Spending (£)'
                   },
                   markers=True)

# Customize the line chart appearance
fig_line.update_traces(line=dict(width=3), marker=dict(size=7))
fig_line.update_layout(xaxis=dict(tickmode='linear'), yaxis=dict(title='Value (£)'))

# Show the line chart
fig_line.show()



In [ ]:
# Compare UK Finance Services Online and UK Abroad Spending 2019 - 2023 ----------------- ?????????????????????????? Not Same as above

import pandas as pd
import plotly.express as px

# Data for UK Finance Services Online Abroad Total
services_online_data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK_Finance_Services_Online_Abroad_Total': [1.40e+10, 1.53e+10, 1.63e+10, 1.52e+10, 1.57e+10]
}

# Data for uk_abroad_total_for_uk_finance
uk_abroad_data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK_Abroad_Total_Spending': [4.277365e+10, 4.389336e+10, 4.415398e+10, 3.782894e+10, 4.095468e+10]
}

# Create dataframes
df_services_online = pd.DataFrame(services_online_data)
df_uk_abroad = pd.DataFrame(uk_abroad_data)

# Merge dataframes on 'year'
df = pd.merge(df_services_online, df_uk_abroad, on='year')

# Create the line chart
fig_line = px.line(df, 
                   x='year', 
                   y=['UK_Finance_Services_Online_Abroad_Total', 'UK_Abroad_Total_Spending'], 
                   title='UK Finance Services Online and UK Abroad Spending 2019 - 2023',
                   labels={'year': 'Year', 
                           'UK_Finance_Services_Online_Abroad_Total': 'Services Online Abroad (£)', 
                           'UK_Abroad_Total_Spending': 'UK Abroad Spending (£)'},
                   markers=True)  # Add markers at each data point

# Customize the line chart appearance
fig_line.update_traces(line=dict(width=3), marker=dict(size=7))
fig_line.update_layout(xaxis=dict(tickmode='linear'), yaxis=dict(title='Value (£)'))

# Show the line chart
fig_line.show()

In [ ]:
# Calculate UK Finance Abroad Online Goods Total

import pandas as pd

# Load the CSV files
df_services = pd.read_csv("UK_Finance_Services_Abroad_Online_Spending_Total.csv")
df_total = pd.read_csv("UK_Finance_Abroad_Total.csv")

# Merge the dataframes on the 'year' column
df_merged = pd.merge(df_total, df_services, on='year')

# Calculate UK Finance Goods Abroad Online Total
df_merged['UK_Finance_Goods_Abroad_Online_Total'] = (
    df_merged['UK_Finance_Abroad_Total'] - df_merged['UK_Finance_Services_Abroad_Online_Spending_Total']
)

# Display the resulting DataFrame
df_merged[['year', 'UK_Finance_Abroad_Total', 'UK_Finance_Services_Abroad_Online_Spending_Total', 'UK_Finance_Goods_Abroad_Online_Total']]

# Save to CSV for later use
df_merged.to_csv('UK_Finance_Goods_Abroad_Online_Total.csv', index=False)

In [ ]:
# Line Chart UK Finance Abroad Totals (Overall, Services Online, Goods Online)

import pandas as pd
import plotly.express as px

# Load the CSV file
file_path = "UK_Finance_Goods_Abroad_Online_Total.csv"
df = pd.read_csv(file_path)

# Create the line chart
fig = px.line(
    df,
    x='year',
    y=[
        'UK_Finance_Abroad_Total',
        'UK_Finance_Services_Abroad_Online_Spending_Total',
        'UK_Finance_Goods_Abroad_Online_Total'
    ],
    title='UK Finance Abroad Totals (Overall, Services Online, Goods Online)',
    labels={
        'year': 'Year',
        'UK_Finance_Abroad_Total': 'Total Finance Abroad (£)',
        'UK_Finance_Services_Abroad_Online_Spending_Total': 'Services Abroad Online (£)',
        'UK_Finance_Goods_Abroad_Online_Total': 'Goods Abroad Online (£)'
    },
    markers=True
)

# Customize the chart appearance
fig.update_traces(line=dict(width=3), marker=dict(size=6))
fig.update_layout(xaxis=dict(tickmode='linear'), yaxis_title='Amount (£)')

# Show the chart
fig.show()



In [ ]:
# UK Finance Abroad Online Spending (Total, Goods, and Services) 2019-2023 ------------- ??????????????????? Differ from Above

import pandas as pd
import plotly.express as px

# Data for UK Finance Abroad Online Total
abroad_online_total_data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK_Abroad_Online_Total': [3.05e+10, 3.67e+10, 3.65e+10, 2.41e+10, 2.51e+10]
}

# Data for UK Finance Abroad Online Goods Total
abroad_online_goods_data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK_Finance_Goods_Online_Abroad_Total': [1.65e+10, 2.14e+10, 2.02e+10, 8.93e+09, 9.45e+09]
}

# Data for UK Finance Abroad Online Services Total
abroad_online_services_data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK_Finance_Services_Online_Abroad_Total': [1.40e+10, 1.53e+10, 1.63e+10, 1.52e+10, 1.57e+10]
}

# Create dataframes for each data set
df_online_total = pd.DataFrame(abroad_online_total_data)
df_online_goods = pd.DataFrame(abroad_online_goods_data)
df_online_services = pd.DataFrame(abroad_online_services_data)

# Merge dataframes on 'year'
df = pd.merge(df_online_total, df_online_goods, on='year')
df = pd.merge(df, df_online_services, on='year')

# Create the line chart
fig_line = px.line(df, 
                   x='year', 
                   y=['UK_Abroad_Online_Total', 'UK_Finance_Goods_Online_Abroad_Total', 'UK_Finance_Services_Online_Abroad_Total'], 
                   title='UK Finance Abroad Online Spending (Total, Goods, and Services) 2019-2023',
                   labels={'year': 'Year', 
                           'UK_Abroad_Online_Total': 'Abroad Online Total (£)', 
                           'UK_Finance_Goods_Online_Abroad_Total_Spending': 'Goods Online Total (£)', 
                           'UK_Finance_Services_Online_Abroad_Total': 'Services Online Total (£)'},
                   markers=True)  # Add markers at each data point

# Customize the line chart appearance
fig_line.update_traces(line=dict(width=3), marker=dict(size=7))
fig_line.update_layout(xaxis=dict(tickmode='linear'), yaxis=dict(title='Value (£)'))

# Show the line chart
fig_line.show()